### IAF mean form 

In [2]:
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import os
from glob import glob
from collections import OrderedDict
import xarray as xr
import numpy as np
import esmlab
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import cartopy
import cartopy.crs as ccrs
import seawater as sw
from seawater.library import T90conv
from scipy import stats
import pop_tools 

from pint import UnitRegistry
import utils
import ncar_jobqueue
from dask.distributed import Client

units = UnitRegistry()
units.define('micromoles_per_kilogram = umol / kg') # used in WOA datasets

Cannot write to data cache folder '/glade/p/cesmdata/cseg'. Will not be able to download remote data files. Use environment variable 'CESMDATAROOT' to specify another directory.


In [3]:
cluster = ncar_jobqueue.NCARCluster(project='ncgd0011')
cluster.adapt(minimum_jobs=0, maximum_jobs=30)
client = Client(cluster)
client

Client Scheduler: tcp://10.12.205.30:44060 Dashboard: https://jupyterhub.ucar.edu/dav/user/kristenk/proxy/46501/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [4]:
#fdas

### MONTHLY

#### Define CESM case and variable

In [25]:
case = 'cesm22cocco.G1850ECOIAF.T62_g17.004'
user = 'kristenk'
#path = '/glade/scratch/kristenk/archive/' + case + '/ocn/proc/tseries/month_1'
path = '/glade/campaign/cgd/oce/projects/besome/cesm-cases/' + case + '/ocn/proc/tseries/month_1'
variables = ['calcToSed_ALT_CO2']
#variables = ['PD']

#### Get last IAF cycle

In [26]:
%%time
ds_list = []
print(f'reading {case}')
ds = xr.Dataset()

for v in variables:
    
    print(v)
    
    files = sorted(glob(f'{path}/{case}.pop.h.{v}.024901-031012.nc'))       
    #files = sorted(glob(f'{path}/{case}.pop.h.{v}.018701-024812.nc'))
    
    
    dsv=xr.open_mfdataset(files, data_vars="minimal", coords='minimal', compat="override", parallel=True, concat_dim="time",
                       drop_variables=["transport_components", "transport_regions"], decode_times=True, chunks={'z_t' : 1})

    ds = xr.merge((ds, dsv))
    ds_list.append(ds)

reading cesm22cocco.G1850ECOIAF.T62_g17.004
calcToSed_ALT_CO2
CPU times: user 130 ms, sys: 29.9 ms, total: 160 ms
Wall time: 6.29 s


In [27]:
coords = {'x':'TLONG','y':'TLAT'}
keepthese=['z_t','time_bound','TAREA','REGION_MASK'] + variables
keep_vars = keepthese +list(coords.values())+['dz','KMT']
ds = ds.drop([v for v in ds.variables if v not in keep_vars])

In [28]:
%%time 
cesm_ann_ds=ds.mean(dim='time').compute()

CPU times: user 316 ms, sys: 180 ms, total: 496 ms
Wall time: 5.44 s


In [29]:
for v in variables:
    
    keep_vars = ['z_t','time_bound','TAREA','time','dz','KMT', v]  + list(coords.values())
    
    ds_out = cesm_ann_ds.drop([v for v in cesm_ann_ds.variables if v not in keep_vars])
    
    #outfile='/glade/scratch/kristenk/archive/'+case+'/ocn/proc/mean4thIAF/mean_IAF4_'+v+'.nc'
    outfile='/glade/scratch/kristenk/archive/'+case+'/ocn/proc/mean5thIAF/mean_IAF5_'+v+'.nc'
    
    ds_out.to_netcdf(outfile)

In [8]:
# v = variables[0]
# outfile='/glade/scratch/kristenk/archive/'+case+'/ocn/proc/mean5thIAF/mean_IAF5_'+v+'.nc'
# cesm_ann_ds.to_netcdf(outfile)

In [14]:
# for v in variables:
#     outfile='/glade/scratch/kristenk/archive/'+case+'/ocn/proc/mean5thIAF/mean_IAF5_'+v+'.nc'
#     cesm_ann_ds[v].to_netcdf(outfile)

### ANNUAL

In [43]:
case = 'cesm22cocco.G1850ECOIAF.T62_g17.010'
#path = '/glade/scratch/kristenk/archive/' + case + '/ocn/proc/tseries/year_1'
path = '/glade/campaign/cgd/oce/projects/besome/cesm-cases/' + case + '/ocn/proc/tseries/year_1'
variables = ['CaCO3_ALT_CO2_FLUX_IN']

In [44]:
%%time

ds_list = []

print(f'reading {case}')

ds = xr.Dataset()

for v in variables:
    
    print(v)
    
    files = sorted(glob(f'{path}/{case}.pop.h.ecosys.nyear1.{v}.0249-0310.nc'))

    dsv=xr.open_mfdataset(files, data_vars="minimal", coords='minimal', compat="override", parallel=True, concat_dim="time",
                       drop_variables=["transport_components", "transport_regions"], decode_times=True)

    
    ds = xr.merge((ds, dsv))
    ds_list.append(ds)

reading cesm22cocco.G1850ECOIAF.T62_g17.010
CaCO3_ALT_CO2_FLUX_IN
CPU times: user 463 ms, sys: 109 ms, total: 572 ms
Wall time: 31.8 s


In [45]:
coords = {'x':'TLONG','y':'TLAT'}
keepthese=['z_t','z_t_150m','time_bound','TAREA','REGION_MASK'] + variables
keep_vars = keepthese +list(coords.values())+['dz','KMT']
ds = ds.drop([v for v in ds.variables if v not in keep_vars])

In [46]:
ds=ds.mean(dim='time')

In [47]:
for v in variables:
        outfile='/glade/scratch/kristenk/archive/'+case+'/ocn/proc/mean5thIAF/mean_IAF5_'+v+'.nc'
        ds[v].to_netcdf(outfile)